In [18]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
import csv
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from sklearn.tree import DecisionTreeRegressor
from datetime import datetime, date, time,timedelta

In [2]:
class classGetData():
    def __init__(self,fileloc):
        self.loc = fileloc
        print('Init...')
        

    def readCSV(self):
        #data1 = open('./data/test.csv')
        print('Reading...')
        file_open = open(self.loc)
        datareader = csv.reader(file_open,delimiter= ',')
        data_arr = []
        for row in datareader:
            data_arr.append(row)

        arr = np.array(data_arr)
        #arr = arr[1:,:]
        print('Done...')
        return arr

In [3]:
def read_data(loc_in):
    a = classGetData(loc_in)
    data1 = a.readCSV()
    return data1

In [4]:
from datetime import datetime
s1 = '10:04:00'
s2 = '11:03:11' # for example
format = '%H:%M:%S'
time = datetime.strptime(s2, format) - datetime.strptime(s1, format)
print (time)

0:59:11


# Training data

In [5]:
#'''Daily questionnaires'''
questionnaires = './que.csv'
raw = read_data(questionnaires)
raw_data = raw[1:,1:]
title = raw[0,1:]
X_que = raw_data.copy()
X_que = X_que.astype(float)

#'''Initial setting'''
init = './init.csv' # morning, afternoon, evening, wake, sleep
raw = read_data(init)
raw_data = raw[1:,:]

X_init = raw_data.copy()
#X_init = X_init.astype(float)

#'''Reminder Ranking wrt daily questionnaires'''
notifi = './notification_lvl.csv'
raw = read_data(notifi)
raw_data = raw[1:,1:]

y_notifi = raw_data.copy()
y_notifi = y_notifi.astype(float)

Init...
Reading...
Done...
Init...
Reading...
Done...
Init...
Reading...
Done...


# Training Validation Test split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X_que[:,:3], y_notifi, test_size=0.2, random_state=42)


In [7]:
def evaluation(y_predict, y_test):
    print("Mean squared error: %.2f" % mean_squared_error(y_test, y_predict))
    # Explained variance score: 1 is perfect prediction
    print('Variance score: %.2f' % r2_score(y_test, y_predict))

# Notification is a classification problem:
## Decision Tree Classification

In [8]:
import graphviz
from sklearn.metrics import accuracy_score

In [10]:
clf = tree.DecisionTreeClassifier()
clf = clf.fit(X_train.ravel().reshape(-1,1), y_train.ravel().reshape(-1,1))

y_dec_predict = clf.predict(X_test.ravel().reshape(-1,1))

In [12]:
dot_data = tree.export_graphviz(clf, out_file=None,
                                class_names=['Level 1','Level 2','Level 3','Level 4','Level 5'], filled=True, 
                                rounded=True,special_characters=True) 
graph = graphviz.Source(dot_data) 
graph.render("decision") 
graphviz.Source(dot_data).view()
print('Accuracy {:.2f}%'.format(100*accuracy_score(y_test.ravel(), y_dec_predict)))

Accuracy 100.00%


# SVM

In [13]:
from sklearn import svm

clf = svm.SVC(gamma='scale',decision_function_shape='ovo').fit(X_train.ravel().reshape(-1,1), y_train.ravel().reshape(-1,1))
svm_predict = clf.predict(X_test.ravel().reshape(-1,1))
print('Accuracy {:.2f}%'.format(100*accuracy_score(y_test.ravel(), svm_predict)))
print('\n')
# Kernels
kern = ['linear', 'rbf', 'sigmoid']

for k in kern:
    clf = svm.SVC(gamma='scale',decision_function_shape='ovo', kernel = k).fit(X_train.ravel().reshape(-1,1), y_train.ravel().reshape(-1,1))
    svm_predict = clf.predict(X_test.ravel().reshape(-1,1))
    print('kernel ', k)
    print('Accuracy {:.2f}%'.format(100*accuracy_score(y_test.ravel(), svm_predict)))
    print('\n')


Accuracy 100.00%


kernel  linear
Accuracy 100.00%


kernel  rbf
Accuracy 100.00%


kernel  sigmoid
Accuracy 61.11%




D:\Program\Anaconda\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Program\Anaconda\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Program\Anaconda\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
D:\Program\Anaconda\envs\py36\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of 

# Nofitication Time¶

In [14]:
mean_X = np.mean(X_que, axis = 0)
print(mean_X)
print(X_init)

[ 11.  -4.  -1. -19.  -9.]
[['7:30' '12:00' '18:00' '7:00' '10:30']]


# Generate notification time

In [20]:
#X_init[0,0]
import datetime
year = '19'
month = '01'
day = '01'
time = year+'/'+month+'/'+day+ ' ' + X_init[0,0]
init_morning = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,1]
init_afternoon = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,2]
init_evening = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,3]
init_wake = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

time = year+'/'+month+'/'+day+ ' ' + X_init[0,4]
init_sleep = datetime.datetime.strptime(time, "%y/%m/%d %H:%M")

init = [init_morning,init_afternoon,init_evening,init_wake,init_sleep]


In [21]:
notification_time = []
for i,t in enumerate(init):
    notification_time.append((t+timedelta(minutes = mean_X[i])).time() )# days, seconds, then other fields.

In [22]:
res = []
for i in range(len(notification_time)):
    res.append(notification_time[i])

print(res[0])
print(res[1])
print(res[2])
print(res[3])
print(res[4])

07:41:00
11:56:00
17:59:00
06:41:00
10:21:00
